# Running the model locally

Before running the following cells, make sure that you've already downloaded the pretrained models inside ```../pretrained_models/```

In [ ]:
%config Completer.use_jedi=False

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")
from io import BytesIO

In [ ]:
import base64
import requests
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from network.Transformer import Transformer

from tqdm import tqdm_notebook
from test_from_code import transform

In [ ]:
styles = ["Hosoda", "Hayao", "Shinkai", "Paprika"]

models = {}

for style in tqdm_notebook(styles):
    model = Transformer()
    model.load_state_dict(torch.load(os.path.join("./../pretrained_models/", style + '_net_G_float.pth')))
    model.eval()
    models[style] = model

In [ ]:
### change this path to test one of the four models locally
path = "../../test-images/paris.jpg"

In [ ]:
img = Image.open(path)
img

In [ ]:
### pick a style in : ["Hosoda", "Hayao", "Shinkai", "Paprika"]
style = "Hosoda"

In [ ]:
### choose a load_size, the higher the better are the results, but the longer is the transformation
load_size = 450

In [ ]:
%%time
output300 = transform(models, style, path, load_size)

In [ ]:
output300

In [ ]:
%%time
### load_size to 450
output450 = transform(models, style, path, load_size=450)

In [ ]:
output450

In [ ]:
%%time
### load_size to 650
output650 = transform(models, style, path, load_size=650)

In [ ]:
output650

## Use the deployed API

In this section, you'll have to put in your API URL

In [ ]:
path = '../../test-images/lawrence.jpg'

#path = "/Users/ahmedbesbes/Downloads/peinture.JPG"

img = Image.open(path)
with open(path, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

url = "https://tbuxfdm545.execute-api.eu-west-3.amazonaws.com/dev/transform"

data = {
    "image": encoded_string,
    "model_id": 1,
    "load_size": 800
}

In [ ]:
img

In [ ]:
%%time
response = requests.post(url, json=data)

In [ ]:
image = response.json()["output"]

image = image[image.find(",")+1:]
dec = base64.b64decode(image + "===")

image = Image.open(BytesIO(dec))

In [ ]:
image